In [20]:
%pip install -qu langchain-openai langchain-community pypdf tavily-python youtube_search langchain-groq

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\AI WAHAJ\Wahaj BixByte\Dawid\.conda\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\AI WAHAJ\Wahaj BixByte\Dawid\.conda\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\AI WAHAJ\Wahaj BixByte\Dawid\.conda\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\AI WAHAJ\Wahaj BixByte\Dawid\.conda\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\AI WAHAJ\Wahaj BixByte\Dawid\.conda\python.exe -m pip install [options] <archive url/path> ...

no such option: -u


IMPORTING LIBARIES

In [21]:
# import Libraries

import openai
import langchain
import pinecone 
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


LOADING ENVIROMENTAL VARIABLES

In [22]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
import os

READING DOCUMENT HELPER FUNCTION

In [24]:
## Lets Read the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [25]:
%pip -q install pypdf

Note: you may need to restart the kernel to use updated packages.


In [26]:
doc=read_doc('PDFs/')
len(doc)

132

CHUNKING TECHNIQUE FOR DOCUMENTS

In [27]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [28]:
documents=chunk_data(docs=doc)
len(documents)

132

SETTING UP PINECONE

In [29]:

import os
import time
import getpass
from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

SETTING UP PINECONE INDEX

In [30]:
import time

index_name = "ytpdfchat"  

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region=os.getenv("PINECONE_ENV")),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

SETTING UP OPENAI EMBEDDINGS

In [31]:
import getpass

os.environ["OPENAI_API_KEY"]= os.getenv("OPENAI_API_KEY")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

SETTING UP VECTOR STORE

In [32]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

ADDING DOCUMENTS TO VECTORSTORE

In [33]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['17c8bcfb-5359-421f-abeb-2daeaabab94a',
 'f0f33994-d51c-42a9-9cc5-34268a3f55fe',
 '96eb43d7-7289-4e52-a969-a8b9fb8d91cd',
 'c7b178cd-87bd-4c76-92f2-f7ff74dbd587',
 '9b46cd7e-0dc6-470b-a54a-191ff30a7608',
 '31ce209b-59f3-41ce-b852-9587fcd03e2c',
 'dd1ccc89-78c0-4d73-a388-24c0e59cb877',
 '994802bc-d867-4c07-9b7f-7ed40fb4f91f',
 'd9485583-e1e3-42b0-b773-2abb80ee4f2d',
 '4cbf2713-2982-4217-acfa-30c73b55b8fd',
 '1a9a9825-6577-4bee-bb7e-dff2ad021de6',
 'f9bdf2b3-d472-4f57-8109-7ac8dc9d3f45',
 '89ff6153-f7ea-4212-995c-6cab2a16cf40',
 '992380d9-279d-40da-889d-d1708ee2159d',
 '7a26a35b-77c8-46e4-99da-b1197de272ce',
 '875a7161-f50d-489b-a94e-16c54a99f974',
 '41fdcd31-71ed-4234-a726-7fdf5f1d48e1',
 'ba50f317-c835-4905-96bc-33bf83490951',
 '068a97cf-8a20-4d5d-9c3a-a0e1dd53c5af',
 'edc787b0-cbd4-4f2b-a859-3c11cee9ffec',
 '484a808b-89e7-49d8-ae52-92b9bc99b541',
 'd21fd868-8de6-4747-b41b-63ceff19377b',
 'd77b8930-3b70-4f5d-bbfe-d6075e07b95b',
 '27584fc2-134f-4d6a-8c4e-94354fc5934c',
 'c8e6e841-31d2-

In [34]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
retriever.invoke("Recent AI advancements", filter={"source": "news"})

[Document(id='0bd68fd7-c95f-4ccd-b4ea-bde71aabdcc8', metadata={'page': 1.0, 'source': 'PDFs\\Research_Paper_On_Artificial_Intelligence_And_Its.pdf'}, page_content='Journal of Advanced Zoology   \n \nAvailable online at: https://jazindia.com                                                                                                                       230  technologies have matured to the point in offering real practical benefits in many of their applications. Major \nartificial intelligence areas are Expert sys tems, Intelligent computer aided instructions, Natural language \nprocessing, Speech understanding, Robotics and sensory systems, Computer vision and scene recognition, \nNeural computing. From these expert system is a rapidly growing technology which is heavi ng a huge impact \non various field of life. The various techniques applied in artificial intelligence are Neural network, Fuzzy \nlogic, Evolutionary computing, Computer aided instructions and Hybrid artificial inte

SETTING UP TAVILY CLIENT


In [35]:
%pip install tavily-python

SETTING UP TAVILY TOOL

In [71]:
from langchain_community.tools import TavilySearchResults

# Initialize the TavilySearchResults tool with custom settings
tavirly_search_tool = TavilySearchResults(
    max_results=5,  # Limit the number of search results
    search_depth="advanced",  # Use advanced search depth
    include_answer=True,  # Include an answer to the query in the results
    include_raw_content=True,  # Include the raw content from the search results
    include_images=True,  # Include image results
    name="Tavily_search",  # Custom name for the tool
    description="This tool searches the web using Tavirly and returns up to 5 results with advanced search depth, including raw content and images."  # Custom description
)


In [72]:
# Perform a search query
response = tavirly_search_tool.invoke({"query": "Recent AI advancements"})
print(response)


[{'url': 'https://www.ibm.com/think/insights/artificial-intelligence-trends', 'content': 'Here are some important current AI trends to look out for in the coming year. Reality check: more realistic expectations. Multimodal AI. Small (er) language models and open source advancements. GPU shortages and cloud costs. Model optimization is getting more accessible. Customized local models and data pipelines.'}, {'url': 'https://www.technologyreview.com/2024/01/08/1085096/artificial-intelligence-generative-ai-chatgpt-open-ai-breakthrough-technologies', 'content': '10 Breakthrough Technologies\nby Will Douglas Heaven\nShare\nPopular\nDeep Dive\nArtificial intelligence\nGoogle DeepMind used a large language model to\xa0solve an unsolved math problem\nThey had to throw away most of what it produced but there was gold among the garbage.\n Was the buzz worth it?\nStay connected\nGet the latest updates fromMIT Technology Review\nDiscover special offers, top stories,\nupcoming events, and more.\n Th

SETTING UP YOUTUBE SEARCH TOOL

In [73]:
from langchain_community.tools import YouTubeSearchTool

# Initialize the YouTubeSearchTool
youtube_search_tool = YouTubeSearchTool(
    name="yotube_search",  # Custom name for the tool
    description="This tool searches YouTube for relevant videos based on a query and returns video URLs.",  # Custom description
)

# Example of running the tool to search for videos
response = youtube_search_tool.invoke("lex fridman, 5")  # "lex fridman" with a limit of 5 results
print(response)


['https://www.youtube.com/watch?v=DyoVVSggPjY&pp=ygULbGV4IGZyaWRtYW4%3D', 'https://www.youtube.com/watch?v=Q8Qk_3a3lUw&pp=ygULbGV4IGZyaWRtYW4%3D', 'https://www.youtube.com/watch?v=Kbk9BiPhm7o&pp=ygULbGV4IGZyaWRtYW4%3D', 'https://www.youtube.com/watch?v=s1oTH4Sjvzg&pp=ygULbGV4IGZyaWRtYW4%3D', 'https://www.youtube.com/watch?v=oFtjKbXKqbg&pp=ygULbGV4IGZyaWRtYW4%3D']


SETTING UP GROQ

In [74]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [76]:
from langchain_groq import ChatGroq

code_interpreter = ChatGroq(
    name="chat_groq",
    model="llama3-groq-70b-8192-tool-use-preview",
    temperature=0,
    max_tokens=8000,  # You can adjust this as needed
    timeout=60,  # Set appropriate timeout based on complexity
    max_retries=2,
    # other params...
)


In [77]:
tools = [youtube_search_tool.as_tool(),tavirly_search_tool.as_tool(),code_interpreter.as_tool(arg_types={"input": str}) ]

In [78]:
print(tools[0].name)
print(tools[1].name)
print(tools[2].name)



yotube_search
Tavily_search
chat_groq


In [67]:
from langchain.prompts import PromptTemplate
prompt_template = """
You are an advanced AI agent with access to multiple tools and a vector database. Your task is to:
1. Understand the user's query.
2. Retrieve relevant information from the vector database if needed.
3. Use the appropriate tool to either search the web, find videos, or interpret code based on the context.
4. Synthesize the results and generate a final response that combines the vector store results and tool outputs.

**Steps**:
- If the query involves a specific technology or content search, query the vector database for related documents.
- If the query mentions video, search YouTube using the YouTube Search tool.
- If the query involves finding general information or web content, use the Tavily Search tool.
- If the query involves a technical request such as code interpretation or debugging, use the Code Interpreter tool.

Remember to combine all relevant information in a cohesive response.

**Examples**:
- For a query like "What are the recent advancements in AI?", query the vector database for related documents, then use Tavily Search to find any additional web content.
- For a query like "Show me recent videos about AI ethics," use the YouTube Search tool to find relevant videos.
- For a query like "Debug the following code...", invoke the Code Interpreter tool to assist with the query.

Now, proceed with the user's query.
"""



In [69]:

from langchain.agents import AgentType, initialize_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    prompt=prompt_template,
    vector_store=vector_store,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,
)

In [82]:
# Example query
query = "What are the recent advancements in AI?"

# Running the agent with the initialized prompt and query
response = agent.run(query)

# Output the response
print("Agent Response:")
print(response)




> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'functions[0].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'functions[0].name', 'code': 'invalid_value'}}